# Shared Local Memory - SLM

This notebook introduces a mechanism that enables data exchange between threads in GPU. By using Shared Local Memory (SLM), often located close to the computing cores, it is possible to do it efficiently. 

## Setup

Load libraries and extensions.

In [ ]:
import pyopencl as cl
import numpy as np
import sys
sys.path.append("..")

from helpers import profile_gpu, profile_cpu, plot, overlay_plots

%load_ext pyopencl.ipython_ext

Create context and queue.

In [ ]:
platform = cl.get_platforms()[0]

ctx = cl.Context(
    dev_type=cl.device_type.ALL, 
    properties=[(cl.context_properties.PLATFORM, platform)])    

queue = cl.CommandQueue(ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)
    
devices = ctx.get_info(cl.context_info.DEVICES)
for d in devices:
    print(f"device={d}")

## Shared local memory

Shared Local Memory (SLM) is memory type that is placed closer to the processors performing operations. It is faster than global memory but usually there is not much of it available. You have to find your hardware specifications to get the actual numbers. 

In [ ]:
%%HTML
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQhMbskzhgt11jxSyvHDHnlD1pfTgUXIj2-_E0w905j5WxXRPBcl96NUKd7EY1uNzz-KvvyhOEPgS2D/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

Shared Local Memory can be a scarce resource and should be used thoughtfully. If used excessively it could have negative effect on the performance. As with all optimizations, resource usage is a balancing act.

Fast SLM is located near the hardware executing work groups (on older Intel integrated GPUs it was located further from blocks performing calculations so it was quite slow). This hardware does not have access to other work groups. Through SLM information can be shared only between threads from the same Work Group Clusters (Streaming Multiprocessors or Subslices).

Typical use cases:
* caching access to global memory
* storing and exchanging offsets or partial calculations
* storing frequently accessed lookup tables

In OpenCL SLM can be allocated in 2 ways:
* statically from within the kernel - this way you have to specify the size of SLM at the compile time. In your kernel:

In [ ]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void kernel_with_static_slm(const __global float *buffer)
{
    __local float localBuffer[1024];
    
    // some other code
}

* dynamically - create a special LocalMemory type object in your host code, specifying the size of memory in bytes.

In [ ]:
threads_amount = 128
local_size_in_bytes = 4 * threads_amount

shared_local_memory = cl.LocalMemory(size=local_size_in_bytes)

This object is then passed together with other arguments in the host code.

In the kernel it's accessed through function arguments with **__local** qualifier.

In [ ]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void kernel_with_dynamic_slm(const __global float *globalbuffer, __local float *localBuffer)
{   
    const uint gid = get_global_id(0);
    int some_value = localBuffer[gid];
    // some other code
}

# Exercise: Derivative Computation

In this exercise we will write a code that computes a simple derivative. We will do it in two iterations. 
* first implement derivative computations using global memory
* then transform the kernel to use Shared Local Memory


A short recap from the university maths gives the following formula for derivative computations.

$$
y' = \lim_{h \to 0} \frac{f(n + h) - f(n)}{h}
$$

So we compute values of the function at point x and at point very close to x. Then we subtract them and divide by change in x. Essentially what is computed is a rate of change on Y-axis to change in X-axis.

This formula would take the following form in sofware developers language:

$$
y' = \frac{y[n + 1] - y[n]}{x[n + 1] - x[n]}
$$

Let's create points for some function - a cosine function was chosen semi-arbitrarily. It is a well know function and has the advantage that we know it's derviative, so we can compare the results computed in GPU with the actual values of the derivative. You can find more information about the derivation [here](https://www.cuemath.com/calculus/derivative-of-cosx/).

* there is cosine function implemented in OpenCL, so we can call instrinsic function

So we have the following formulas to work with:
$$
y = \cos(x)
\newline
y' = -\sin(x)
$$

For nice visualization let's use a few periods. If you wish you can later experiments with different functions.

In [ ]:
step = np.float32(1e-4)
x = np.arange(0, 8 * np.pi, step).astype(np.float32)
f = lambda x: np.cos(x)
y = f(x)
print(f'We have {len(x)}, points to process.')

Plot the function.

In [ ]:
plot(x, y, 'x', 'cos(x)')

Since we know the derivative of the function we can also plot it. We will expect similar plot for points computed in GPU.

In [ ]:
f_prime = lambda x: -np.sin(x)
y_prime = f_prime(x)
plot(x, y_prime, 'x', '-sin(x)')

### Task: Calculate the derviate in GPU

In order to perform the calculation in GPU we need to do the usual OpenCL setup. Create output buffer for datapoints of the derivative.

In [ ]:
flags = cl.mem_flags
d_x = cl.Buffer(ctx, flags.READ_ONLY | flags.COPY_HOST_PTR, hostbuf=x)
d_y_prime = cl.Buffer(ctx, flags.WRITE_ONLY, y.nbytes)

Your task is to fix the kernel below to calculate the derviate according to the formula given above. So you need to:
* compute function points at points x_1 and x_0
* subtract the neighbouring values of f(x)
* divide the difference by step
* write the value back to the output buffer

In [ ]:
%%cl_kernel

float f(float x)
{
    return cos(x);
}

__kernel void compute_derivative(const __global float *x,
                                 __global float *y_prime,
                                 int nr_elements)
{
    int thread_id = get_global_id(0);
    int grid_stride = get_global_size(0);
    
    for(int i = thread_id; i < nr_elements; i+=grid_stride) {
        float x_0 = x[i];
        float x_1 = x[i + 1];
        
        // Fix the code below
            
        y_prime[i] = 0;        
    }    
}

Set some execution configuration. Threads in this exercise are reused, meaning that each will handle multiple points, looping through them using grid-stride pattern.

In [ ]:
local_work_size = (64,)
global_work_size = (local_work_size[0] * 128,)

nr_output_elements = np.int32(len(x) - 1)

print(f'local threads = {local_work_size[0]}, global threads = {global_work_size[0]}, {nr_output_elements} points to process.')

Schedule the execution with profiling.

In [ ]:
_ = profile_gpu(compute_derivative, 20,
            queue, 
            global_work_size, 
            local_work_size,
            d_x,
            d_y_prime,
            nr_output_elements
            )

Fetch the data back from GPU. The plots should be overlaid and only the blue one should be visible.

In [ ]:
def fetch_data(queue, d_buffer, nr_elements):
    h_result = np.zeros(nr_elements + 1).astype(np.float32)
    cl.enqueue_copy(queue, h_result, d_buffer)
    
    return h_result

def compare(ground_truth, result, error_tolerance = 1e-3):
    s = 0
    e = len(result)

    np.testing.assert_allclose(ground_truth[s:e], result[s:e], rtol=error_tolerance, atol=error_tolerance)

h_y_prime_from_gpu = fetch_data(queue, d_y_prime, nr_output_elements)

overlay_plots(x, y_prime, h_y_prime_from_gpu)

compare(y_prime, h_y_prime_from_gpu)

Refer to the [solution](./compute_derivative_global_memory.c) if you get stuck.

### Performance constrains - Compute Bound vs Memory Bound

There are two high level views on the performance bottlenecks which limit the execution latency:
* Memory speed
* Computation speed

If our program is limited by accesses to the memory we say that it is "Memory Bound". In that case speeding up computations will not make any observable improvement to the time it takes to execute a kernel. Some examples of techniques to reduce memory load can be:
* access memory is coelesced manner - which will utilize cache efficiently 
* explicitely cache data into SLM
* access memory with large data types - eg. use uint4 to load data instead of four times uint
* watch register usage - so how many variables you have declared in your kernel. If you exceed maximum number of registers available per thread, the compiler is forced to move the data to slower memory type and the performance hit is significant.

If the program is limited by computations we say it is "Compute Bound". That can be the case for complex algorithms requiring lot of mathematical or binary operations. In that case data is loaded quickly but it takes a lot of GPU clocks to process it. Some examples of ways to tackle it are:
* spread the computations between threads
* avoid duplicating computations in each thread - use SLM to exachange computed data
* use lookup tables for data that can be pre computed
* use intrinsics if possible - functions provided with your framework

# Synchronization and barriers

Before diving into the code it's neccessary to explain a few key GPU concepts:
* thread synchronization
* shared local memory

In [ ]:
%%HTML
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQg7xzKDqvmAbPtl735zXtHc4itBbi7ar9jd6hHw9xt3zIsdBfSAna_DePFQa27TdeEyljFVKB3CEG_/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

Work groups usually consist of multiple sub groups. All threads within a subgroup execute the same instruction. However  subgroups, are not automatically synchronized with each other. They execute on different processors.

More complex algorithms require some communication between threads - they require data calculated in one thread to be used in some other thread.

Threads can exchange information through following mechanisms:
* global memory
* shared local memory
* non-standard vendor specific extensions

When one thread writes to a memory location and other threads reads from that place, we are exposed to a possible race condition. The value of that memory cell will depend on the order in which reading and writing threads accessed it. This problem should be familiar to you from virtually any multithreading programming environment.

It's important to understand that there are only 2 kinds of explicit synchronization possible in GPU:
* work group level synchronization - synchronizes threads from all sub groups within a group. When a barrier is placed somewhere in a kernel, threads within that group will wait at the barrier until all of them reach that execution point. At that place in a code we are sure that all operations until the barrier have completed - including operations writing to memory intended for other threads. To place a [barrier](https://www.khronos.org/registry/OpenCL/sdk/1.0/docs/man/xhtml/barrier.html) in your kernel code write:


    - barrier(CLK_LOCAL_MEM_FENCE) - use with Shared Local Memory
    - barrier(CLK_GLOBAL_MEM_FENCE) - use with global memory
    - barrier(CLK_LOCAL_MEM_FENCE | CLK_GLOBAL_MEM_FENCE) - both


* grid level synchronization - there is no way to synchronize work groups in a grid from kernel code. Work groups can be distributed across different Streaming multiprocessors (on Nvidia - groups of cores) or Subslices (on Intel - groups of Execution Units which group hardware threads - SIMDs - executing multiple software threads). The only way to synchronize work groups is through scheduling a separate kernel. 

### Task: Convert the kernel to use shared memory.

Your next task is to convert the kernel computing function derivate to use shared local memory.

The kernels we've written so far are Memory Bound. To demonstrate the effect of using SLM in such simple example we need to simulate complex computations - so we will artifically force the kernel to be Compute Bound. Calculating cosine function is very fast so actually using SLM in this case does not make sense, since each thread can quickly calculate neighboring values. However if we force the code to be compute bound (it's performance will be limited by computational complexity), then it makes sense to share data which is already computed by other thread. For that purpose function f was modified with a for-loop to take relatively significant amount of time to compute.

As in the previous example, the code below computes the values of function f twice:
* once for it's own thread
* once for neighbouring thread
However, since computing f is slow now this there is a performance hit.

Your job is to:
* add SLM to the kernel
* Optimize the kernel to compute the value of f only once per thread
* Share the results with the neighbouring threads.
* don't forget to syncronize threads in a appropriate place

This should speed up execution times by a factor of two.

In [ ]:
%%cl_kernel

// This function is deliberately made inefficient to demonstrate some complex computations
float f(float x)
{
    float y = 0;
    // make sure the result is just cos(x) but slow the execution down
    for (int i = -100; i < 1; i++)
    {
        y = cos(x + i);
    }
    return y;
}


__kernel void compute_derivative_slm(const __global float *x, 
                                     __global float *y_prime, 
                                     int nr_elements)
{
    const int lid = get_local_id(0);
    const int local_size = get_local_size(0) - 1;
    const int num_groups = get_num_groups(0);
    const int group_id = get_group_id(0);
    
    const int grid_stride = num_groups * local_size;
    const int thread_id = lid  + group_id * local_size;

    for(int i = thread_id; i < nr_elements; i+=grid_stride) {
        float x_0 = x[i];
        float x_1 = x[i + 1];

        
        float y_0 = f(x_0);
        float y_1 = f(x_1);
        
        if (lid < local_size)
        {
            y_prime[i] = (y_1 - y_0) / (x_1 - x_0);
        }
    }
}

Setup and schedule execution.

In [ ]:
x = np.arange(0, 256 * 64 * np.pi, step).astype(np.float32)

local_work_size = (64,)
global_work_size = (local_work_size[0] * 128,)

nr_output_elements = np.int32(len(x) - 1)

print(f'local threads = {local_work_size[0]}, global threads = {global_work_size[0]}, {nr_output_elements} points to process.')

d_x = cl.Buffer(ctx, flags.READ_ONLY | flags.COPY_HOST_PTR, hostbuf=x)
d_y_prime = cl.Buffer(ctx, flags.WRITE_ONLY, x.nbytes)

profile_gpu(compute_derivative_slm, 20,
            queue, 
            global_work_size, 
            local_work_size,
            d_x,
            d_y_prime,
            nr_output_elements
            )

h_y_prime_from_gpu = fetch_data(queue, d_y_prime, nr_output_elements)

# plot and compare part of the result
s = 0
e = int(1000 * np.pi * 64)
plot(x[s:e], h_y_prime_from_gpu[s:e], 'x', 'derivative (x)')

y_prime = f_prime(x)
compare(y_prime[s:e], h_y_prime_from_gpu[s:e])

NOTE for inquisitive readers: the way to calculate grid_stride and thread_id was changed with respect to previous exercise. They both depend on ```local_size``` which is shrink by one thread to
```local_size = get_local_size(0) - 1```.

The reason for that is that we need to handle derivative calculation at the edge of work groups. To calculate derviate we need one more function value point than the actual size of work group - each thread need two points. We can tackle it the following ways:
* skip last thread when computing derviative - and make following work-groups start one entry earlier
* make one thread calculate function value twice

The first option was chosen and it makes just one thread idle while the seconds makes mutliple threads idle.

Refer to the [solution](./compute_derivative_solution_slm.c) if you get stuck.

# Advanced Exercise: Parallel Patterns - Reduction

Reduction is very common parallel algorithm. It combines elements of an array into one element, using the same two-inputs one-output associative operator. 

Associative operator gives the same results no matter what the order of operations is.

$$
(a + b) + c  = a + (b + c) 
$$

Examples or reductions of an array include:
* calculating sum of all elements
* finding maximum/minimum value
* calculating value of a polynomial at a point

In [ ]:
%%HTML
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vSv_IJe5in4xdBK1hLiXJ0idEZTl2S2nNsaQNCNgbPa6ES8u5k-M3AxrKjcMH6BVG3Q0VQnMLjjCr94/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

### Reduction - summing in CPU

To illustrate the reducion performance in host code we will have a look at summing array elements.

Below are examples of summing elements of a collection in three ways:
* serially - written in python 'by hand'
* using build in 'sum'
* using numpy sum which underneath the hood uses native compiled code

NOTE: There is no GPU acceleration in numpy

In [ ]:
import numpy as np

N=1024 * 256
buffer = np.arange(0, N).astype(np.float32)

def serial_sum(buffer):
    value = 0
    for i in buffer:
        value += i
    
    return value
    
run_count = 10
_ = profile_cpu(serial_sum, run_count, buffer)
_ = profile_cpu(sum, run_count, buffer)
_ = profile_cpu(np.sum, run_count, buffer)

1. Which method is the fastest?
2. Are the results consistent between all the methods? Why? What to do about it?

## Reduce implementation in GPU

In [ ]:
%%HTML
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQc4ae3fOz73VpylkfLm-I9xvOfGGU0cTjRG2tXiLMp5iPdQJ4lrxem9-XUAWOpycpZW06YVrU1Vqqw/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

## Reduction Exercise 

Below is "work in progress" code which performs partial reduction in GPU. Each work group calculates a sum of all cell corresponding to this thread. The resulting sums are summed up in CPU.

You task is to take over the code and improve it.
1. Correct the code so that the kernel calculates partial sums correctly.
    * You need to fix race condition - analyze reads and writes to temporary memory
2. Use all available memory types - convert the kernel to use SLM:
    * covert access IDs
    * convert the barrier type
    * allocate different memory type in host code
3. Further optimize the code's performance:
    * utilize threads effectively - make subgroups busy or idle as in the slides above
    * make each thread busy at first iteration - currently first thread only loads data to temporary buffer
   
Notice how performance differs with the changes you make.

For simplicity assume that the size of input data is always mulitple of work group size.

In [ ]:
%%cl_kernel -o "-cl-fast-relaxed-math"

// Task 2 - consider all available memory types
__kernel void reduce(const __global int *in_buf, __global int *temp, __global int *result)
{
    const uint gid = get_global_id(0);
    const uint group_id = get_group_id(0);
    const int local_size = get_local_size(0);
    const int local_id = get_local_id(0);
    
    // Task 2 - what ids should be used to store and later retrieve intermediate results?
    temp[gid] = in_buf[gid];

    // Task 3 - Is this looping most convenient for calculations?
    for (int step = 1; step < local_size; step *= 2)
    {   
        // Task 1 - fix synchronization
        if (gid % (step * 2) == 0) // Task 3 - think which threads perform the actual addition
        {
            temp[gid] = temp[gid] + temp[gid + step];
        }        
    }
    
    if (local_id == 0) {
        result[group_id] = temp[gid];
    }
}

Prepare the data. For simplicity of reasoning we will:
* Fill the input buffer with ones, so effectively the reduction will calculate number of elements
* Have one thread per one buffer element (subject to optimization)

Please keep in mind that we do not handle integer precision overflow.

In [ ]:
buffer_type = np.int32
N=67108864
M=128
global_work_size = (N, )
local_work_size = (M, )
num_groups = N // M

h_buffer = np.ones(N).astype(buffer_type)

print (f"local_work_size = {local_work_size} global_work_size = {global_work_size} Nr Work Groups = {num_groups}")

Create input and output buffers. Please note that the algorithm requires writing data back to some intermediate buffer. As a starting point it will be global memory.

In [ ]:
flags = cl.mem_flags
d_input_buffer = cl.Buffer(ctx, flags.READ_ONLY | flags.COPY_HOST_PTR, hostbuf=h_buffer)

# Task 2 - is this memory the best for sharing data between threads?
d_intermediate_buffer = cl.Buffer(ctx, flags.READ_WRITE, h_buffer.nbytes)

d_result = cl.Buffer(ctx, flags.WRITE_ONLY, 4 * num_groups)

Schedule kernel execution and fetch the data back.

In [ ]:
gpu_time_ms = profile_gpu(reduce, 50,
                          queue, 
                          global_work_size, 
                          local_work_size,
                          d_input_buffer,
                          d_intermediate_buffer,
                          d_result
                          )

h_result = np.zeros(num_groups).astype(np.int32)
_ = cl.enqueue_copy(queue, h_result, d_result)

Compare the results with values computed by numpy. Please note that results computed in GPU are partial. There are multiple solutions to tackle that:
* schedule another kernel to compute the remaining sum of sums
* compute remaining work in CPU

Since the purpose of this exercise is to demonstrate the usage of SLM we will go for just summing the remining values in CPU. We are going to profile this sum as well to see if the overall hybrid CPU + GPU solutions still makes sense from latency perspective.

In [ ]:
res_opencl, sum_partials_ms = profile_cpu(np.sum, 20, h_result)

res_numpy, numpy_ms = profile_cpu(np.sum, 20, h_buffer)

np.testing.assert_equal(res_opencl, res_numpy)
print (f"gpu   reduction summed value = {res_opencl} median time took {gpu_time_ms:.4f} ms, sum partial results took {sum_partials_ms:.4f} ms")
print (f"numpy reduction summed value = {res_numpy} median time took {numpy_ms:.4f} ms.")

Questions:
* How does execution time change with every optimization?
* Which optimizations were beneficial?
* Can you explain why there are differences?

If you get stuck:
* [Part 1 solution](./reduction_part1_solution.c)
* [Part 2 solution](./reduction_part2_solution.c)
* [Part 3 solution](./reduction_part3_solution.c)

## Further optimizations

In [ ]:
%%HTML
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQhdPev2THHUBZD8ORyBFgZwOT_2ST-yQe17Zv00eIkWQkS9U1rQNXgXmsXMzJLL9L301r1tkhlqrZQ/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

You are welcome to experiment with this and other optimization on a other platforms - perhaps on your own machine.

## Curiosities

* In OpenCL 2.0 there was a kernel function added [work_group_reduce_add](https://www.khronos.org/registry/OpenCL/sdk/2.1/docs/man/xhtml/work_group_reduce.html). As the same suggests it does what our reduce kernel does. There is actually a whole family of reduce functions.

* A bunch of other useful work group level functions were added - you can read about it [here](https://software.intel.com/en-us/articles/using-opencl-20-work-group-functions).

* For floating point operations order matters - so it is not assosiative operation
* There is a limited precision in 32 bits numbers. In case of large sum this can quickly overflow.

## Extra task

Convert reduction kernel to use intrinsic function [work_group_reduce_add](https://www.khronos.org/registry/OpenCL/sdk/2.1/docs/man/xhtml/work_group_reduce.html) to perform reduction. Observe further reduction in execution time and how the code is simplified.

If you get stuck solution is below.

* [Intrinsics solution](./reduction_intrinsic_solution.c)